In [ ]:
from aht_fitbit import main 

vars = main.get_vars()

vol = vars['vol']

In [ ]:
import pyspark.sql.functions as f
import re

def getMaxDatefromDir(dir):
  df = spark.createDataFrame(dbutils.fs.ls(dir))
  _max = df.select(f.max(df.name)).collect()[0]['max(name)'] 
  pattern = r'[^0-9]'
  # Match all digits in the string and replace them with an empty string
  strMax = re.sub(pattern, '', _max)
  date_obj = dt.datetime.strptime(strMax, '%Y%m%d').date()
  return date_obj,_max

In [ ]:
sleep_dir = f"dbfs:{vol}/raw_fitbitapi/sleep"
act_dir = f"dbfs:{vol}/raw_fitbitapi/activities"
actLogs_dir = f"dbfs:{vol}/raw_fitbitapi/activitylog"